In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# import shutil

# source_folder = '/content/drive/MyDrive/cyclegans/preprocess'
# destination_folder = '/content/drive/MyDrive/cyclegans_new/preprocess'

# shutil.copytree(source_folder, destination_folder)


'/content/drive/MyDrive/cyclegans_new/preprocess'

In [ ]:
# !mkdir -p '/content/drive/MyDrive/cyclegans_new/preprocess'

In [ ]:
# !rm -r /content/drive/MyDrive/cyclegans_new


In [18]:
# # Change this path to where you want your project to live in Google Drive
# %cd /content/drive/MyDrive/cyclegans_new

# # Clone the repo and install dependencies
# !git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix.git
# %cd pytorch-CycleGAN-and-pix2pix/
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 19.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 108.3 MB/s eta 0:00:00
  Created wheel for visdom: filename=visdom-0.2.4-py3-none-any.whl size=1408195 sh

In [17]:
%cd /content/drive/MyDrive/cyclegans_new/pytorch-CycleGAN-and-pix2pix

/content/drive/MyDrive/cyclegans_new/pytorch-CycleGAN-and-pix2pix


# PERCEPTUAL LOSS
To improve the visual quality of the generated images, we supplement the standard L1 cycle-consistency loss with a Perceptual Loss (also known as VGG Loss). While L1 loss is effective at preserving the overall structure, it often encourages overly smooth or blurry results by averaging pixel values. The Perceptual Loss addresses this by comparing high-level features (like textures and edges) instead of raw pixels. It uses a pre-trained VGG19 network as an expert feature extractor. By minimizing the difference between the feature maps of the generated and target images, the model is encouraged to produce significantly sharper and more realistic details that better align with human perception.

In [8]:
# Read the existing networks.py file
with open('models/networks.py', 'r') as f:
    original_content = f.read()

# Define the new VGGPerceptualLoss class
perceptual_loss_code = """
import torch
import torch.nn as nn
from torchvision import models

class VGGPerceptualLoss(nn.Module):
    def __init__(self, resize=True):
        super(VGGPerceptualLoss, self).__init__()
        blocks = []
        blocks.append(models.vgg19(pretrained=True).features[:4].eval())
        blocks.append(models.vgg19(pretrained=True).features[4:9].eval())
        blocks.append(models.vgg19(pretrained=True).features[9:18].eval())
        blocks.append(models.vgg19(pretrained=True).features[18:27].eval())
        for bl in blocks:
            for p in bl.parameters():
                p.requires_grad = False
        self.blocks = nn.ModuleList(blocks)
        self.transform = nn.functional.interpolate
        self.resize = resize
        self.register_buffer("mean", torch.tensor([0.485, 0.456, 0.406]).view(1, 3, 1, 1))
        self.register_buffer("std", torch.tensor([0.229, 0.224, 0.225]).view(1, 3, 1, 1))

    def forward(self, input, target):
        # VGG expects 3 channels. If input is grayscale, repeat the channel.
        if input.shape[1] != 3:
            input = input.repeat(1, 3, 1, 1)
            target = target.repeat(1, 3, 1, 1)
        # De-normalize from [-1, 1] to [0, 1]
        input = (input + 1) / 2
        target = (target + 1) / 2
        # Normalize for VGG
        input = (input - self.mean) / self.std
        target = (target - self.mean) / self.std
        if self.resize:
            input = self.transform(input, mode='bilinear', size=(224, 224), align_corners=False)
            target = self.transform(target, mode='bilinear', size=(224, 224), align_corners=False)
        loss = 0.0
        x = input
        y = target
        for block in self.blocks:
            x = block(x)
            y = block(y)
            loss += nn.functional.l1_loss(x, y)
        return loss
"""

# Append the new class to the original content and overwrite the file
with open('models/networks.py', 'w') as f:
    f.write(original_content + "\n" + perceptual_loss_code)

print("✅ Appended VGGPerceptualLoss to models/networks.py")
print("Note: The first time you train, PyTorch will download the VGG19 model weights.")

✅ Appended VGGPerceptualLoss to models/networks.py
Note: The first time you train, PyTorch will download the VGG19 model weights.


In [9]:
%%writefile models/cycle_gan_model.py
import torch
import itertools
from util.image_pool import ImagePool
from .base_model import BaseModel
from . import networks


class CycleGANModel(BaseModel):
    @staticmethod
    def modify_commandline_options(parser, is_train=True):
        parser.set_defaults(no_dropout=True)
        if is_train:
            parser.add_argument('--lambda_A', type=float, default=10.0, help='weight for cycle loss (A -> B -> A)')
            parser.add_argument('--lambda_B', type=float, default=10.0, help='weight for cycle loss (B -> A -> B)')
            parser.add_argument('--lambda_identity', type=float, default=0.5, help='use identity mapping.')
            # Add a new flag for our perceptual loss
            parser.add_argument('--lambda_perceptual', type=float, default=1.0, help='weight for perceptual loss')
        return parser

    def __init__(self, opt):
        BaseModel.__init__(self, opt)
        # Add 'perceptual' to the list of losses to log
        self.loss_names = ['D_A', 'G_A', 'cycle_A', 'idt_A', 'D_B', 'G_B', 'cycle_B', 'idt_B', 'perceptual']
        visual_names_A = ['real_A', 'fake_B', 'rec_A']
        visual_names_B = ['real_B', 'fake_A', 'rec_B']
        self.visual_names = visual_names_A + visual_names_B
        self.model_names = ['G_A', 'G_B', 'D_A', 'D_B']
        self.netG_A = networks.define_G(opt.input_nc, opt.output_nc, opt.ngf, opt.netG, opt.norm,
                                        not opt.no_dropout, opt.init_type, opt.init_gain, self.gpu_ids)
        self.netG_B = networks.define_G(opt.output_nc, opt.input_nc, opt.ngf, opt.netG, opt.norm,
                                        not opt.no_dropout, opt.init_type, opt.init_gain, self.gpu_ids)

        if self.isTrain:
            self.netD_A = networks.define_D(opt.output_nc, opt.ndf, opt.netD,
                                            opt.n_layers_D, opt.norm, opt.init_type, opt.init_gain, self.gpu_ids)
            self.netD_B = networks.define_D(opt.input_nc, opt.ndf, opt.netD,
                                            opt.n_layers_D, opt.norm, opt.init_type, opt.init_gain, self.gpu_ids)
            self.fake_A_pool = ImagePool(opt.pool_size)
            self.fake_B_pool = ImagePool(opt.pool_size)
            # define loss functions
            self.criterionGAN = networks.GANLoss(opt.gan_mode).to(self.device)
            self.criterionCycle = torch.nn.L1Loss()
            self.criterionIdt = torch.nn.L1Loss()
            # Initialize the perceptual loss
            self.criterionPerceptual = networks.VGGPerceptualLoss().to(self.device)
            # initialize optimizers
            self.optimizer_G = torch.optim.Adam(itertools.chain(self.netG_A.parameters(), self.netG_B.parameters()), lr=opt.lr, betas=(opt.beta1, 0.999))
            self.optimizer_D = torch.optim.Adam(itertools.chain(self.netD_A.parameters(), self.netD_B.parameters()), lr=opt.lr, betas=(opt.beta1, 0.999))
            self.optimizers.append(self.optimizer_G)
            self.optimizers.append(self.optimizer_D)

    def set_input(self, input):
        AtoB = self.opt.direction == 'AtoB'
        self.real_A = input['A' if AtoB else 'B'].to(self.device)
        self.real_B = input['B' if AtoB else 'A'].to(self.device)
        self.image_paths = input['A_paths' if AtoB else 'B_paths']

    def forward(self):
        """Run forward pass"""
        self.fake_B = self.netG_A(self.real_A)
        self.rec_A = self.netG_B(self.fake_B)
        self.fake_A = self.netG_B(self.real_B)
        self.rec_B = self.netG_A(self.fake_A)

    def backward_D_basic(self, netD, real, fake):
        pred_real = netD(real)
        loss_D_real = self.criterionGAN(pred_real, True)
        pred_fake = netD(fake.detach())
        loss_D_fake = self.criterionGAN(pred_fake, False)
        loss_D = (loss_D_real + loss_D_fake) * 0.5
        loss_D.backward()
        return loss_D

    def backward_D_A(self):
        fake_B = self.fake_B_pool.query(self.fake_B)
        self.loss_D_A = self.backward_D_basic(self.netD_A, self.real_B, fake_B)

    def backward_D_B(self):
        fake_A = self.fake_A_pool.query(self.fake_A)
        self.loss_D_B = self.backward_D_basic(self.netD_B, self.real_A, fake_A)

    def backward_G(self):
        lambda_idt = self.opt.lambda_identity
        lambda_A = self.opt.lambda_A
        lambda_B = self.opt.lambda_B
        lambda_perceptual = self.opt.lambda_perceptual

        # Identity loss
        self.loss_idt_A = 0
        self.loss_idt_B = 0
        if lambda_idt > 0:
            self.idt_A = self.netG_A(self.real_B)
            self.loss_idt_A = self.criterionIdt(self.idt_A, self.real_B) * lambda_B * lambda_idt
            self.idt_B = self.netG_B(self.real_A)
            self.loss_idt_B = self.criterionIdt(self.idt_B, self.real_A) * lambda_A * lambda_idt

        # GAN loss
        self.loss_G_A = self.criterionGAN(self.netD_A(self.fake_B), True)
        self.loss_G_B = self.criterionGAN(self.netD_B(self.fake_A), True)
        # Cycle loss
        self.loss_cycle_A = self.criterionCycle(self.rec_A, self.real_A) * lambda_A
        self.loss_cycle_B = self.criterionCycle(self.rec_B, self.real_B) * lambda_B

        # --- NEW: Perceptual Loss Calculation ---
        # Compare real and reconstructed images in both directions
        self.loss_perceptual_A = self.criterionPerceptual(self.rec_A, self.real_A) * lambda_A * lambda_perceptual
        self.loss_perceptual_B = self.criterionPerceptual(self.rec_B, self.real_B) * lambda_B * lambda_perceptual
        self.loss_perceptual = self.loss_perceptual_A + self.loss_perceptual_B
        # --- END NEW ---

        # Combine all losses and calculate gradients
        self.loss_G = (self.loss_G_A + self.loss_G_B + self.loss_cycle_A + self.loss_cycle_B +
                       self.loss_idt_A + self.loss_idt_B + self.loss_perceptual)
        self.loss_G.backward()

    def optimize_parameters(self):
        self.forward()
        self.set_requires_grad([self.netD_A, self.netD_B], False)
        self.optimizer_G.zero_grad()
        self.backward_G()
        self.optimizer_G.step()
        self.set_requires_grad([self.netD_A, self.netD_B], True)
        self.optimizer_D.zero_grad()
        self.backward_D_A()
        self.backward_D_B()
        self.optimizer_D.step()

print("✅ Patched cycle_gan_model.py with Perceptual Loss.")

Overwriting models/cycle_gan_model.py


#Custom Dataloaders

In [10]:
%%writefile data/unaligned_npy_dataset.py
import os
import torch
import numpy as np
from data.base_dataset import BaseDataset

class UnalignedNpyDataset(BaseDataset):
    """This dataset class can load unaligned/unpaired datasets of .npy files."""
    def __init__(self, opt):
        BaseDataset.__init__(self, opt)
        self.dir_A = os.path.join(opt.dataroot, 'trainA')
        self.dir_B = os.path.join(opt.dataroot, 'trainB')

        self.A_paths = sorted([os.path.join(self.dir_A, f) for f in os.listdir(self.dir_A) if f.endswith('.npy')])
        self.B_paths = sorted([os.path.join(self.dir_B, f) for f in os.listdir(self.dir_B) if f.endswith('.npy')])
        self.A_size = len(self.A_paths)
        self.B_size = len(self.B_paths)

    def __getitem__(self, index):
        A_path = self.A_paths[index % self.A_size]
        index_B = np.random.randint(0, self.B_size - 1)
        B_path = self.B_paths[index_B]

        A_npy = np.load(A_path).astype(np.float32)
        B_npy = np.load(B_path).astype(np.float32)

        A_tensor = torch.from_numpy(A_npy.transpose((2, 0, 1)))
        B_tensor = torch.from_numpy(B_npy.transpose((2, 0, 1)))

        return {'A': A_tensor, 'B': B_tensor, 'A_paths': A_path, 'B_paths': B_path}

    def __len__(self):
        return max(self.A_size, self.B_size)

Overwriting data/unaligned_npy_dataset.py


In [11]:
import re

# Read the existing networks.py file
with open('models/networks.py', 'r') as f:
    original_content = f.read()

# Remove the old, buggy VGGPerceptualLoss class definition if it exists
# This makes the script safe to run multiple times
cleaned_content = re.sub(r'class VGGPerceptualLoss.*', '', original_content, flags=re.DOTALL)

# Define the new, corrected VGGPerceptualLoss class
corrected_perceptual_loss_code = """
import torch
import torch.nn as nn
from torchvision import models

class VGGPerceptualLoss(nn.Module):
    def __init__(self, resize=True):
        super(VGGPerceptualLoss, self).__init__()
        blocks = []
        blocks.append(models.vgg19(pretrained=True).features[:4].eval())
        blocks.append(models.vgg19(pretrained=True).features[4:9].eval())
        blocks.append(models.vgg19(pretrained=True).features[9:18].eval())
        blocks.append(models.vgg19(pretrained=True).features[18:27].eval())
        for bl in blocks:
            for p in bl.parameters():
                p.requires_grad = False
        self.blocks = nn.ModuleList(blocks)
        self.transform = nn.functional.interpolate
        self.resize = resize
        self.register_buffer("mean", torch.tensor([0.485, 0.456, 0.406]).view(1, 3, 1, 1))
        self.register_buffer("std", torch.tensor([0.229, 0.224, 0.225]).view(1, 3, 1, 1))

    def forward(self, input, target, feature_layers=[0, 1, 2, 3], style_layers=[]):
        # --- START OF FIX ---
        # Handle inputs with different channel numbers correctly
        if input.shape[1] == 1: # Grayscale
            input = input.repeat(1, 3, 1, 1)
            target = target.repeat(1, 3, 1, 1)
        elif input.shape[1] == 2: # 2-Channel SAR Data
            # Use the first channel (e.g., VV) and repeat it to create a 3-channel grayscale image
            input = input[:, 0:1, :, :].repeat(1, 3, 1, 1)
            target = target[:, 0:1, :, :].repeat(1, 3, 1, 1)
        # --- END OF FIX ---

        # De-normalize from [-1, 1] to [0, 1]
        input = (input + 1) / 2
        target = (target + 1) / 2
        # Normalize for VGG
        input = (input - self.mean) / self.std
        target = (target - self.mean) / self.std
        if self.resize:
            input = self.transform(input, mode='bilinear', size=(224, 224), align_corners=False)
            target = self.transform(target, mode='bilinear', size=(224, 224), align_corners=False)
        loss = 0.0
        x = input
        y = target
        for i, block in enumerate(self.blocks):
            x = block(x)
            y = block(y)
            if i in feature_layers:
                loss += nn.functional.l1_loss(x, y)
        return loss
"""

# Append the new class to the cleaned content and overwrite the file
with open('models/networks.py', 'w') as f:
    f.write(cleaned_content + "\n" + corrected_perceptual_loss_code)

print("✅ Corrected the VGGPerceptualLoss class in models/networks.py")

✅ Corrected the VGGPerceptualLoss class in models/networks.py


In [12]:
%%writefile util/util.py
"""This module contains simple helper functions """
from __future__ import print_function
import torch
import numpy as np
from PIL import Image
import os


def tensor2im(input_image, imtype=np.uint8):
    """"Converts a Tensor array into a numpy image array."""
    if not isinstance(input_image, np.ndarray):
        if isinstance(input_image, torch.Tensor):
            image_tensor = input_image.data
        else:
            return input_image
        image_numpy = image_tensor[0].cpu().float().numpy()

        # --- START OF FIX ---
        # Handle inputs with different channel numbers correctly for visualization
        if image_numpy.ndim == 2:  # Handle 2D arrays
            image_numpy = np.expand_dims(image_numpy, axis=0)
        if image_numpy.shape[0] == 1:  # Grayscale
            image_numpy = np.tile(image_numpy, (3, 1, 1))
        elif image_numpy.shape[0] == 2:  # 2-Channel SAR
            image_numpy = np.tile(image_numpy[0:1, :, :], (3, 1, 1))
        # --- END OF FIX ---

        image_numpy = (np.transpose(image_numpy, (1, 2, 0)) + 1) / 2.0 * 255.0
    else:
        image_numpy = input_image
    return image_numpy.astype(imtype)


def save_image(image_numpy, image_path, aspect_ratio=1.0):
    """Save a numpy image to the disk"""
    image_pil = Image.fromarray(image_numpy)
    h, w, _ = image_numpy.shape
    if aspect_ratio > 1.0:
        image_pil = image_pil.resize((h, int(w * aspect_ratio)), Image.BICUBIC)
    if aspect_ratio < 1.0:
        image_pil = image_pil.resize((int(h / aspect_ratio), w), Image.BICUBIC)
    image_pil.save(image_path)


def mkdirs(paths):
    """create empty directories if they don't exist"""
    if isinstance(paths, list) and not isinstance(paths, str):
        for path in paths:
            mkdir(path)
    else:
        mkdir(paths)


def mkdir(path):
    """create a single empty directory if it doesn't exist"""
    if not os.path.exists(path):
        os.makedirs(path)

Overwriting util/util.py


#TRAINING

In [ ]:
!python train.py \
  --dataroot /content/drive/MyDrive/cyclegans_new/preprocess/sar_to_rgb/ \
  --name sar2rgb_perceptual \
  --model cycle_gan \
  --netG resnet_9blocks \
  --dataset_mode unaligned_npy \
  --input_nc 2 \
  --output_nc 3 \
  --no_flip \
  --lambda_identity 0 \
  --lr 0.0001 \
  --lambda_perceptual 1.0
  --continue_train \
  --epoch_count 11



✅ Dataloader files for .npy format have been created/patched.
✅ Patched cycle_gan_model.py with Perceptual Loss.
----------------- Options ---------------
               batch_size: 1                             
                    beta1: 0.5                           
          checkpoints_dir: ./checkpoints                 
           continue_train: True                          	[default: False]
                crop_size: 256                           
                 dataroot: /content/drive/MyDrive/cyclegans_new/preprocess/sar_to_rgb/	[default: None]
             dataset_mode: unaligned_npy                 	[default: unaligned]
                direction: AtoB                          
              display_env: main                          
             display_freq: 400                           
               display_id: 1                             
            display_ncols: 4                             
             display_port: 8097                          
        